In [16]:
import math

def cnk(n, k):
    ret = 1
    for i in range(k):
        ret *= (n - i)
    for i in range(2, k+1):
        ret /= i
    return ret

def logCnk(n, k):
    if 2 * k > n:
        k = n - k
    ret = 0
    for i in range(k):
        ret += math.log(n-i) - math.log(i+1)
    return ret
def logP(n, p, x):
    ret = logCnk(n,x)
    if p != 0:
        ret += x * math.log(p)
    if 1 - p != 0:
        ret += (n-x) * math.log(1 - p)
    #print("logP", logCnk(n,x), x * math.log(p), (n-x) * math.log(1-p))
    #print("logP", n, p, x, ret)
    return ret
    # return logCnk(n,x) + x * math.log(p) + (n-x) * math.log(1 - p)

def cal_L(nums, n, p):
    ret = 0
    for x in nums:
        ret += logP(n, p, x)
    #print("cal_L", nums, n, p, ret)
    return ret

def get_best_p(s, t, n):
    return s / (t * n)

def check(nums, n, p):
    print("check: n = %d, p = %f, t = %d" % (n, p, len(nums)))
    s = sum(nums)
    t = len(nums)
    max_x = max(nums)
    print("\tcheck_p", s / (n * t), p)
    if s / (n * t) != p:
        p = s / (n * t)
        print("\tupdate p = %f" % p)
    r = 1
    for i in range(-r, r+1, 1):
        if n + i < max_x:
            continue
        print("\tcheck_L" if i != 0 else "\tCHECK_L", n + i, cal_L(nums, n+i, p), cal_L(nums, n+i, get_best_p(s, t, n+i)))
    return nums, n, p

In [27]:
import numpy as np

import random

def check_n(nums, t, p, n):
    n = n + 1
    ret = t * math.log(n * (1 - p))
    for x in nums:
        ret -= math.log(n - x)
    return ret < 0

def m_step(nums, p):
    s = sum(nums)
    t = len(nums)
    n = max(nums)
    for i in range(n, n + 1000):
        if check_n(nums, t, p, i):
            return i
    print("Warning")
    return n + 1000

def get_best_n(nums, p):
    s = sum(nums)
    t = len(nums)
    l = max(nums)
    w = 1
    for i in range(32):
        r = l + w
        if check_n(nums, t, p, r):
            break
        w *= 2
    if not check_n(nums, t, p, r):
        print("Warning")
    while l < r:
        mid = (l + r) // 2
        if check_n(nums, t, p, mid):
            r = mid
        else:
            l = mid + 1
    return l

def get_mid(l, r):
    return l + (r - l) / 3, l + 2 *(r - l) / 3

def get_mid_int(l, r):
    if r - l < 3:
        return l + 1, l + 2
    return l + (r - l) // 3, l + 2 * (r - l) // 3

def solve(nums):
    t = len(nums)
    s = sum(nums)
    
    l = 0.0
    r = 1.0
    for i in range(100):
        p0, p1 = get_mid(l, r)
        #print("init p %f, %f" % (p0, p1))
        #n0, n1 = m_step(nums, p0), m_step(nums, p1)
        n0, n1 = get_best_n(nums, p0), get_best_n(nums, p1)
        p0, p1 = get_best_p(s, t, n0), get_best_p(s, t, n1)
        s0, s1 = cal_L(nums, n0, p0), cal_L(nums, n1, p1)
        #print("upda p %f, %f" % (p0, p1))
        if s0 > s1:
            r = p1
        else:
            l = p0
        if n0 == n1:
            break
    return nums, n0, p0

def build_exp(n, p, t):
    return np.random.binomial(n, p, t)

def test(func, T = 10):
    for i in range(T):
        N = random.randint(1, 1000)
        P = random.uniform(0.1, 0.9)
        T = random.randint(50, 200)
        X = build_exp(N, P, T)
        print("TEST %d\n" % i, N, P, T, X)
        _, n, p = func(X)
        print("Checking the ground truth.")
        check(X, N, P)
        print("Checking the result.")
        check(X, n, p)
        print("result:\n", "\treal:", N, P, T, "\n\tguest:", n, p)

#test(solve, 10)

x1 = [16,18,22,25,27]
x2 = [16,18,22,25,28]
check(*solve(x1)), check(*solve(x2))


check: n = 99, p = 0.218182, t = 5
	check_p 0.21818181818181817 0.21818181818181817
	check_L 98 -14.18822627973221 -14.181124512028333
	CHECK_L 99 -14.181118773296777 -14.181118773296777
	check_L 100 -14.188125618532663 -14.181132015096559
check: n = 191, p = 0.114136, t = 5
	check_p 0.11413612565445026 0.11413612565445026
	check_L 190 -14.488022982153275 -14.486330281901136
	CHECK_L 191 -14.48633001880037 -14.48633001880037
	check_L 192 -14.488011034577802 -14.48633086543797


(([16, 18, 22, 25, 27], 99, 0.21818181818181817),
 ([16, 18, 22, 25, 28], 191, 0.11413612565445026))